In [1]:
import pandas as pd
import numpy as np


from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException  

from datetime import date
import time
import os
import shutil

In [2]:
#RACES_XP = '/html/body/div[4]/div[3]/div[3]/div[2]/div[2]'
#NAME_RACE_XP = '/html/body/div[4]/div[3]/div[3]/div[2]/div[1]/div[1]/h1'
#NAV_XP = '/html/body/div[4]/div[3]/div[3]/div[2]/div[1]/ul'

#BUTTON_SERIES_XP = '/html/body/div[4]/div[3]/div[3]/div[1]/div/div[1]/div/div[2]/span[1]'
#GRID_SERIES_XP = '/html/body/div[4]/div[3]/div[3]/div[1]/div/div[3]/div[1]/div'

#HREF = 'https://ru.motorsport.com/f1/results'
#CLASS_NAME_SERIES= 'ms-filter-item__title'
#CLASS_NAME_RACE = 'ms-results-standings-header'
#CLASS_NAME_CALENDAR = 'ms-results-subnav'
#CLASS_NAME_CALENDAR_ITEMS = 'ms-results-subnav_item'
#CLASS_NAME_MENU = 'ms-tab-menu'

In [3]:
config = np.load('parser_config.npy', allow_pickle=True).item()

def write_head_table(file: object, table_head:object) -> None:
    
    title_table = []
                
    for col in table_head:
        if col.text: title_table.append(col.text)
        else: title_table.append('None')
    
    file.write(';'.join(title_table) + '\n')


def write_data_table(file:object, strings: object) -> None:
    for string in strings:
        cols = string.find_elements(By.TAG_NAME, 'td')
        data = []
        for col in cols:
            if col.text:
                data.append(col.text.replace('\n', ' '))
            else:
                data.append('None')
        file.write(';'.join(data) + '\n')


def get_links_series(driver: object) -> list:
    
    button = driver.find_element(By.XPATH, config['BUTTON_SERIES_XP'])
    button.click()
    time.sleep(3)
    menu_series = driver.find_element(By.XPATH, config['GRID_SERIES_XP'])
    time.sleep(3)
    series_items = menu_series.find_elements(By.TAG_NAME, 'a')

    links = [elem.get_attribute('href') for elem in series_items]

    return links    


def logging():
    pass

___

TEST

___

In [4]:
#STAGE_1 Launch and setings
#-------------------------------
years = list(range(2015, 2024))
options = Options()
options.add_argument('-headless')
driver = Chrome(options=options)
driver.get(config['HREF'])

series_links = get_links_series(driver)

#STAGE_2 Load series links and their iterations
#-------------------------------
for link in series_links:
    for year in years:
        try:
            work_link = f'{link}{year}'
            driver.get(work_link)
            time.sleep(1)
            name_series = driver.find_element(By.CLASS_NAME, config['CLASS_NAME_SERIES']).text
        except Exception as e:
            continue
        
        
        #создание директории при ее отсутсвии
        current_path = os.getcwd()
        
        main_path = f'{current_path}/data/{name_series}'
        if os.path.exists(main_path) is False: os.mkdir(main_path)
        
        var_path = f'{main_path}/{year}'
        if os.path.exists(var_path) is False: os.mkdir(var_path)
        print('STAGE#3')
    
    
        driver.get(work_link)
        time.sleep(1)

#STAGE_3 Load links of race and their iteration
#-------------------------------
        navig_races = driver.find_element(By.CLASS_NAME, config['CLASS_NAME_CALENDAR'])
        time.sleep(3)
        all_races = navig_races.find_elements(By.CLASS_NAME, config['CLASS_NAME_CALENDAR_ITEMS'])
        time.sleep(3)

        links_race = [elem.get_attribute('href') for elem in all_races]

        print('STAGE#4')
        
        for link_r in links_race:
            if link_r is not None:

                driver.get(link_r)
                time.sleep(1)
                race_name = driver.find_element(By.CLASS_NAME, config['CLASS_NAME_RACE']).text

                
                var_path = f'{main_path}/{year}/{race_name}'
                if os.path.exists(var_path) is False: os.mkdir(var_path)

#STAGE_4 Load links of races event and their iteration
#-------------------------------
                navig_event = driver.find_element(By.CLASS_NAME, config['CLASS_NAME_MENU'])
                time.sleep(1)

                links_part = [elem.get_attribute('href') for elem in navig_event]
                for link_p in links_part:
                    if link is not None:
                        driver.get(link_p)
                        table_name = link_p[link_p.find('st=') + 3:].lower()
                    
                        tables = driver.find_elements(By.TAG_NAME, 'table')
                        changing_name = True if len(tables) > 1 else False

                        #Проверка на наличие данной таблицы
                        #Если таблица уже существует, то не парсим дальше
                        if os.path.exists(result) \
                                and os.stat(result).st_size != 0: continue
            
#STAGE_5 Parsing table(s) and write in raw.txt
#-------------------------------
                        for i, table in enumerate(tables):
                            with open('raw.txt', 'w+') as file:

                                try:
                                    head = table.find_element(By.TAG_NAME, 'thead')
                                    table_head = head.find_elements(By.TAG_NAME, 'th')

                                    write_head_table(file, table_head)
                                except NoSuchElementException:
                                    pass
                    
                                finally:
                                    body = table.find_element(By.TAG_NAME, 'tbody')
                                    strings = body.find_elements(By.TAG_NAME, 'tr')
                
                                    write_data_table(file, strings)
                            
                            result  = f'{var_path}/{table_name}{i}.csv' if changing_name else f'{var_path}/{table_name}.csv'
                            #копирование данных из промежуточного текстового файла в .csv файл в готовую папку
                            shutil.copyfile('raw.txt', result)
                    
                    else:
                        continue
            else:
                continue
            logging()

STAGE#3
STAGE#4


TypeError: 'WebElement' object is not iterable